In [1]:
# from reso_extract import ms_graph
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import sys
from pathlib import Path

# Define the path you want to add
new_path = Path("C:\\Users\\andmn\\PycharmProjects\\kverse")

# Convert Path object to string and append it to sys.path
sys.path.append(str(new_path))
from selenium.webdriver.support.wait import WebDriverWait
from kdags.assets.reparation.reso_old.web_driver import initialize_driver, login_to_reso
from kdags.assets.reparation.reso_old.document_extractor import (
    extract_docs_data,
    click_see_service_order,
)
from kdags.assets.reparation.reso_old.file_processor import process_docs_data
from kdags.assets.reparation.reso_old.web_utils import (
    wait_for_page_load,
    ensure_clickable,
)
from kdags.assets.reparation.reso_old.file_utils import save_file_locally
from kdags.assets.reparation.reso_old.config import DEFAULT_WAIT
import pandas as pd
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import TimeoutException
import time
import os
from datetime import datetime
from kdags.resources.tidyr import DataLake
import polars as pl
import re
import json

driver = initialize_driver()
wait = WebDriverWait(driver, DEFAULT_WAIT)
from urllib.parse import urlparse  # **** ADDED IMPORT ****
import unicodedata
import requests

In [2]:
login_to_reso(driver, wait)

True

In [3]:
def click_presupuesto(driver, wait):
    # ---  Clicking the SPAN containing "Presupuesto" ---
    # Use XPath to find the specific <span> element containing the text
    presupuesto_locator = (By.XPATH, "//span[normalize-space(text())='Presupuesto']")
    # Wait for the span element to be present on the page
    presupuesto_span = wait.until(EC.presence_of_element_located(presupuesto_locator))
    # Scroll the element into view to ensure it's not off-screen
    driver.execute_script("arguments[0].scrollIntoView(true);", presupuesto_span)
    time.sleep(0.5)  # Brief pause after scroll
    wait.until(EC.element_to_be_clickable(presupuesto_locator))
    presupuesto_span.click()  # Click the span
    print("Waiting for loading overlay to disappear...")
    wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR, ".overlay")))
    print("Loading overlay disappeared.")


click_presupuesto(driver, wait)

Waiting for loading overlay to disappear...
Loading overlay disappeared.


In [6]:
def search_service_order(driver, wait, service_order):
    """
    Searches for a specific service order on the page and waits for results.

    Args:
        driver: The Selenium WebDriver instance.
        wait: The WebDriverWait instance.
        service_order: The service order number (int or str) to search for.
    """
    # --- Find the input field, clear it, and enter the service order ---
    print(f"Locating the service order input field (codigoBuscar)...")
    search_input_locator = (By.ID, "codigoBuscar")
    search_input = wait.until(EC.presence_of_element_located(search_input_locator))
    print(f"Entering service order: {service_order}")
    search_input.clear()
    search_input.send_keys(str(service_order))  # Ensure input is string
    # ---------------------------------------------------------------------

    # --- Find and click the search button ---
    print("Locating the search button (buttonBuscar)...")
    search_button_locator = (By.ID, "buttonBuscar")
    search_button = wait.until(EC.element_to_be_clickable(search_button_locator))
    print("Clicking the search button...")
    search_button.click()
    # -----------------------------------------

    # --- Wait for the filtering results (assuming the overlay reappears) ---
    print("Waiting for search results filtering overlay to disappear...")
    overlay_locator = (By.CSS_SELECTOR, ".overlay")
    # IMPORTANT: This assumes the *same* overlay with class 'overlay' is used for filtering.
    wait.until(EC.invisibility_of_element_located(overlay_locator))
    print("Search results filtering complete.")
    # --------------------------------------------------------------------


def click_see_service_order(wait):
    """
    Waits for the results table and clicks the 'See service order' button
    for the first row.

    Args:
        driver: The Selenium WebDriver instance.
        wait: The WebDriverWait instance.
    """
    # --- Wait for the results table ---
    print("Waiting for the results table (WebGrid)...")
    webgrid_locator = (By.ID, "WebGrid")
    wait.until(EC.presence_of_element_located(webgrid_locator))
    print("Results table found.")

    # --- Wait directly for the button to be clickable and click it ---
    # Locator targets the button with class 'verOrdenServicio' in the first row (implicitly)
    see_order_button_locator = (By.CSS_SELECTOR, "button.verOrdenServicio")
    print("Waiting for the 'See service order' button to be clickable...")
    # This wait implicitly handles presence and visibility before checking clickability
    see_order_button = wait.until(EC.element_to_be_clickable(see_order_button_locator))
    print("'See service order' button is clickable.")

    print("Clicking the 'See service order' button...")
    see_order_button.click()  # Using standard click
    print("Clicked 'See service order' button.")
    # ------------------------------------------------------------------


def navigate_to_quotation_tab(driver, wait):
    """
    Waits for and clicks the 'Quotation' tab, then waits for its content to load.

    Args:
        driver: The Selenium WebDriver instance.
        wait: The WebDriverWait instance.
    """
    # --- Wait for the Quotation tab button to be ready and click it ---
    print(
        "Waiting for the 'Quotation' tab button (v-pills-presupuesto-tab) to be clickable..."
    )
    quotation_tab_locator = (By.ID, "v-pills-presupuesto-tab")

    # Wait for the tab button to be clickable
    quotation_tab_button = wait.until(EC.element_to_be_clickable(quotation_tab_locator))
    print("'Quotation' tab button is clickable.")

    print("Clicking the 'Quotation' tab button...")
    quotation_tab_button.click()
    print("Clicked 'Quotation' tab button.")
    # ---------------------------------------------------------------------

    # --- Wait for content within the Quotation tab to load (if necessary) ---
    # Assumes the overlay appears again. Adjust if needed.
    print("Waiting for Quotation tab content to load (overlay to disappear)...")
    overlay_locator = (By.CSS_SELECTOR, ".overlay")
    wait.until(EC.invisibility_of_element_located(overlay_locator))
    print("Quotation tab content loaded (overlay disappeared).")
    # --------------------------------------------------------------------------

In [7]:
def extract_quotation_details(driver, wait, service_order):
    """
    Extracts details from the most recent quotation entry displayed.

    Args:
        driver: The Selenium WebDriver instance.
        wait: The WebDriverWait instance.
        service_order: The service order number associated with this quotation.

    Returns:
        A dictionary containing the extracted quotation details,
        or None if the container is not found.
    """
    # --- Locate the main container for quotations ---
    print("Waiting for Quotation container (divPres1)...")
    quotation_container_locator = (By.ID, "divPres1")
    quotation_container = wait.until(
        EC.visibility_of_element_located(quotation_container_locator)
    )
    print("Quotation container found and visible.")

    # --- Locate the first/most recent entry block within the container ---
    print("Locating the first quotation entry block...")
    first_entry_locator = (By.CSS_SELECTOR, "#divPres1 > div:first-child > div")
    first_entry_element = wait.until(
        EC.visibility_of_element_located(first_entry_locator)
    )
    print("First quotation entry block found and visible.")

    # --- Extract data from the first entry block ---
    quotation_data = {}

    # Version and Purchase Order Status
    version_po_element = first_entry_element.find_element(
        By.CSS_SELECTOR, "div > span > i"
    )
    version_po_text = version_po_element.text
    print(f"Found Version/PO text block: {version_po_text}")
    version_match = re.search(r"Version (\d+):", version_po_text)
    quotation_data["version"] = int(version_match.group(1)) if version_match else None
    po_status_match = re.search(
        r"Purchase Order:\s*([^\n]+)", version_po_text, re.IGNORECASE
    )
    quotation_data["purchase_order_status"] = (
        po_status_match.group(1).strip() if po_status_match else None
    )

    print(f"Extracted Version: {quotation_data['version']}")
    print(f"Extracted PO Status: {quotation_data['purchase_order_status']}")

    # Locate the block containing the most recent details
    details_container = first_entry_element.find_element(By.CSS_SELECTOR, "div > div")
    most_recent_detail_block = details_container.find_element(
        By.CSS_SELECTOR, "div:first-child > div:last-child"
    )

    # Date/Time
    date_block = most_recent_detail_block.find_element(By.CSS_SELECTOR, "i.fecha")
    day = date_block.find_element(By.TAG_NAME, "dia").text
    month = date_block.find_element(By.TAG_NAME, "mes").text
    year_time = date_block.find_element(By.TAG_NAME, "año").text
    quotation_data["date_time_str"] = f"{day} {month} {year_time}".strip()

    # User
    user_element = most_recent_detail_block.find_element(
        By.XPATH, ".//span[contains(normalize-space(), 'User:')]"
    )
    quotation_data["user"] = user_element.text.split("User:")[1].strip()

    # Amount
    amount_element = most_recent_detail_block.find_element(
        By.XPATH, ".//span[contains(normalize-space(), 'Amount:')]"
    )
    quotation_data["amount"] = amount_element.text.split("Amount:")[1].strip()

    # Remarks
    remarks_element = most_recent_detail_block.find_element(
        By.CSS_SELECTOR, "span.overflowText"
    )
    remarks_full_text = remarks_element.text
    quotation_data["remarks"] = (
        remarks_full_text.split("Remarks:")[1].strip()
        if "Remarks:" in remarks_full_text
        else remarks_full_text.strip()
    )

    # Add the service order
    quotation_data["service_order"] = service_order

    # Extract Download URL
    print("Extracting download URL...")
    download_link_element = first_entry_element.find_element(By.TAG_NAME, "a")
    download_url = download_link_element.get_attribute("href")
    quotation_data["download_url"] = download_url
    print(f"Found Download URL: {download_url}")

    print("\nExtracted Quotation Data (within function):")
    print(json.dumps(quotation_data, indent=4))

    return quotation_data

In [8]:
def navigate_to_documents_tab(wait):
    """
    Waits for and clicks the 'Documents' tab, then waits for its content to load.

    Args:
        driver: The Selenium WebDriver instance.
        wait: The WebDriverWait instance.
    """
    # --- Wait for the Documents tab button to be ready and click it ---
    print(
        "Waiting for the 'Documents' tab button (v-pills-documentos-tab) to be clickable..."
    )
    documents_tab_locator = (By.ID, "v-pills-documentos-tab")

    # Wait for the tab button to be clickable
    documents_tab_button = wait.until(EC.element_to_be_clickable(documents_tab_locator))
    print("'Documents' tab button is clickable.")

    print("Clicking the 'Documents' tab button...")
    documents_tab_button.click()
    print("Clicked 'Documents' tab button.")
    # ---------------------------------------------------------------------

    # --- Wait for content within the Documents tab to load (if necessary) ---
    # Assume the overlay appears again. Adjust if needed.
    print("Waiting for Documents tab content to load (overlay to disappear)...")
    overlay_locator = (By.CSS_SELECTOR, ".overlay")
    wait.until(EC.invisibility_of_element_located(overlay_locator))
    print("Documents tab content loaded (overlay disappeared).")
    # You might need to wait for a specific element *within* the Documents tab instead, e.g.:
    # wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#v-pills-documentos .some-element")))
    # --------------------------------------------------------------------------


def extract_document_links(driver, wait):

    documents_list = []
    documents_container_locator = (By.ID, "v-pills-documentos")
    document_link_locator = (By.CSS_SELECTOR, "#v-pills-documentos a.documento")

    # print("Waiting for the documents container to be visible...")
    wait.until(EC.visibility_of_element_located(documents_container_locator))
    # print("Documents container is visible.")

    # print("Waiting for document links to be present...")
    wait.until(EC.presence_of_element_located(document_link_locator))
    # print("Document links are present.")

    document_elements = driver.find_elements(*document_link_locator)
    # print(f"Found {len(document_elements)} document links.")

    if not document_elements:
        # print("No documents found in the panel.")
        return documents_list

    # Extract info for each document
    for i, element in enumerate(document_elements):
        # print(f"Processing document {i+1}...")
        doc_info = {}
        url = element.get_attribute("href")
        doc_info["url"] = url

        # **** ADDED: Extract Filename from URL ****
        file_name = None
        if url:
            try:
                parsed_url = urlparse(url)
                url_path = parsed_url.path
                # Use pathlib to reliably get the last part of the path
                file_name = Path(url_path).name
            except Exception as e:
                print(f"  Error parsing URL to get filename for doc {i+1}: {e}")
                file_name = None  # Ensure it's None if parsing failed

        # **** ADDED: Assertion for filename ****
        assert file_name, f"Could not extract filename from URL for doc {i+1}: {url}"
        doc_info["file_name"] = file_name
        # *****************************************

        file_title = None
        file_subtitle = None

        # Find all direct child spans of the link element
        all_spans = element.find_elements(By.XPATH, "./span")  # Direct child spans

        if all_spans:
            # Title Logic (same as before)
            if all_spans[0].text and "File Title:" not in all_spans[0].text.strip():
                file_title = all_spans[0].text.strip()
            elif (
                len(all_spans) > 1
                and all_spans[1].text
                and "File Title:" not in all_spans[1].text.strip()
            ):
                file_title = all_spans[1].text.strip()

            # Subtitle Logic (same as before)
            for idx, span in enumerate(all_spans):
                span_text = span.text.strip()
                if "File Title:" in span_text and idx + 1 < len(all_spans):
                    file_subtitle = all_spans[idx + 1].text.strip()
                    if file_title is None and idx > 0:
                        file_title = all_spans[idx - 1].text.strip()
                    break

        # Fallback title (same as before)
        if file_title is None:
            full_text = element.text
            lines = [
                line.strip()
                for line in full_text.split("\n")
                if line.strip() and "(See more" not in line
            ]
            file_title = " ".join(lines) if lines else "Unknown Title"

        # Assign to dictionary
        doc_info["file_title"] = (
            file_title.replace("\n", " ").strip() if file_title else None
        )
        doc_info["file_subtitle"] = (
            file_subtitle.replace("\n", " ").strip() if file_subtitle else None
        )

        # print(f"  Title: {doc_info['file_title']}")
        # print(f"  Subtitle: {doc_info['file_subtitle']}")
        # print(f"  Filename: {doc_info['file_name']}")  # Added print
        # print(f"  URL: {doc_info['url']}")

        # Only add if we got a URL (filename assertion already checked)
        documents_list.append(doc_info)

    # print(f"Extracted {len(documents_list)} document records.")
    return documents_list


def normalize_text(text):
    """Converts text to lowercase and removes accents."""
    if not text:
        return ""
    # NFD decomposes characters into base character and combining marks
    # Filtering Mn category removes the combining marks (accents)
    normalized = "".join(
        c for c in unicodedata.normalize("NFD", text) if unicodedata.category(c) != "Mn"
    )
    return normalized.lower()


def filter_report_documents(documents_list):
    """
    Filters a list of document dictionaries based on normalized file_title.

    Keeps documents where the normalized file_title exactly matches one of
    "carta presupuesto", "informe tecnico preliminar", or "informe tecnico final".

    Args:
        documents_list: A list of dictionaries, where each dictionary has
                        at least a 'file_title' key.

    Returns:
        A new list containing only the filtered document dictionaries.
    """
    if not documents_list:
        return []

    target_titles_normalized = {
        "carta presupuesto",
        "informe tecnico preliminar",
        "informe tecnico final",
    }
    filtered_docs = []

    # print(f"Filtering {len(documents_list)} documents...")

    for doc_info in documents_list:
        original_title = doc_info.get("file_title")
        if not original_title:
            # print(
            #     f"  Skipping document with no title: {doc_info.get('file_name', 'N/A')}"
            # )
            continue

        # Normalize the extracted file_title
        # Remove bracketed content like '[Cliente]' or '[Revisado]' before normalizing
        title_no_brackets = re.sub(r"\[.*?\]", "", original_title).strip()
        normalized_title = normalize_text(title_no_brackets)
        # print(
        #     f"  Original Title: '{original_title}' -> Normalized (no brackets): '{normalized_title}'"
        # )

        # Check if the normalized title is in our target set
        if normalized_title in target_titles_normalized:
            # print(f"    MATCH FOUND! Keeping document.")
            filtered_docs.append(doc_info)  # Add the original dictionary

    print(f"Finished filtering. Kept {len(filtered_docs)} documents.")
    return filtered_docs

In [9]:
def copy_reso_documents_to_datalake(documents_list, service_order):
    """
    Downloads documents from HTTPS blob URLs and uploads them to ADLS.

    Args:
        documents_list: List of dictionaries from extract_document_links,
                        each containing at least 'url' and 'file_name'.
        service_order: The service order number to use in the destination path.

    Returns:
        A list of destination URIs for successfully copied files.
    """
    if not documents_list:
        print("No documents provided to copy.")
        return []

    print(f"Starting copy process for Service Order {service_order}...")
    datalake = DataLake()  # Instantiate your DataLake helper
    # Get the underlying Azure DataLakeServiceClient
    # Note: Assumes your DataLake class exposes the client or you can access it.
    # If not, you might need to initialize it here using the connection string.
    # service_client = DataLakeServiceClient.from_connection_string(os.environ["AZURE_STORAGE_CONNECTION_STRING"])
    service_client = (
        datalake.client
    )  # Assuming direct access as per datalake.py structure

    base_container = "bhp-raw-data"
    base_folder = f"RESO/DOCUMENTS/{service_order}"
    target_base_uri = f"abfs://{base_container}/{base_folder}"

    successful_copies = []
    total_files = len(documents_list)

    for i, doc_info in enumerate(documents_list):
        source_url = doc_info.get("url")
        file_name = doc_info.get("file_name")
        print(f"\nProcessing file {i+1}/{total_files}: {file_name}")

        if not source_url or not file_name:
            print(f"  Skipping document {i+1} due to missing URL or filename.")
            continue

        destination_uri = f"{target_base_uri}/{file_name}"
        print(f"  Source: {source_url}")
        print(f"  Target: {destination_uri}")

        try:
            # 1. Download content from source URL
            print(f"  Downloading content...")
            response = requests.get(source_url, timeout=60)  # Added timeout
            response.raise_for_status()  # Check for download errors (4xx or 5xx)
            content = response.content
            print(f"  Download successful ({len(content)} bytes).")

            # 2. Upload content to ADLS destination
            print(f"  Uploading to ADLS...")
            # Parse destination URI
            parsed_dest = urlparse(destination_uri)
            dest_container = parsed_dest.netloc
            dest_path = parsed_dest.path
            if dest_path.startswith("/"):
                dest_path = dest_path[1:]

            # Get file system and file clients
            file_system_client = service_client.get_file_system_client(dest_container)

            # Ensure directory exists (Needed for ADLS Gen2)
            directory_path = str(Path(dest_path).parent).replace(
                "\\", "/"
            )  # Use pathlib for robust parent path
            if directory_path and directory_path != ".":
                directory_client = file_system_client.get_directory_client(
                    directory_path
                )
                directory_client.create_directory()  # Creates if not exists

            file_client = file_system_client.get_file_client(dest_path)

            # Upload data (overwrite if exists)
            file_client.upload_data(content, overwrite=True)
            print(f"  Upload successful.")
            successful_copies.append(destination_uri)

        except requests.exceptions.RequestException as e:
            print(f"  ERROR downloading {file_name}: {e}")
        except Exception as e:
            print(f"  ERROR uploading {file_name} to {destination_uri}: {e}")

    print(
        f"\nFinished copy process. Successfully copied {len(successful_copies)} out of {total_files} files."
    )
    return successful_copies

In [10]:
def close_service_order_view(wait):
    """
    Clicks the 'closePCI' element to close the detailed service order view
    and waits for the search input field on the previous screen to appear.

    Args:
        driver: The Selenium WebDriver instance.
        wait: The WebDriverWait instance.
    """
    # --- Wait for the close button/div to be clickable ---
    print("Waiting for the close button (closePCI) to be clickable...")
    close_button_locator = (By.ID, "closePCI")

    # Wait for the element to be clickable
    close_button = wait.until(EC.element_to_be_clickable(close_button_locator))
    print("Close button (closePCI) is clickable.")

    print("Clicking the close button...")
    close_button.click()
    print("Clicked close button.")
    # ----------------------------------------------------

    # --- Wait for navigation back to the previous screen ---
    # Assuming returning to the search screen means the 'codigoBuscar' input is present again
    print("Waiting to return to the search view (waiting for 'codigoBuscar')...")
    previous_screen_element_locator = (By.ID, "codigoBuscar")
    wait.until(EC.presence_of_element_located(previous_screen_element_locator))
    print("Returned to the search view (found 'codigoBuscar').")
    # You might need to add a wait for an overlay to disappear here as well, if one appears:
    # print("Waiting for overlay to disappear after closing...")
    # wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR, ".overlay")))
    # print("Overlay disappeared.")
    # -----------------------------------------------------

In [13]:
service_orders = pl.read_parquet(
    r"C:\Users\vales\DataspellProjects\workspace\component_status.parquet"
)["service_order"].to_list()

In [16]:
# --- Main Processing Loop ---

# List to store results
quotations_details = (
    []
)  # Renamed for clarity if you store more than just quotation info
processed_docs_info = []  # To potentially store info about copied docs

# **** ADDED: Get total count ****
total_orders = len(service_orders)
print(f"Found {total_orders} service orders to process.")

# **** ADDED: Use enumerate for index and value ****
for i, service_order in enumerate(service_orders):
    # **** MODIFIED: Print progress ****
    print(
        f"\n--- Processing service order {i+1} of {total_orders}: {service_order} ---"
    )

    # --- Steps for each service order ---
    try:  # Optional: Add error handling for the entire SO process
        print(f"  Step 1: Searching for SO...")
        search_service_order(driver, wait, service_order)

        print(f"  Step 2: Clicking 'See Service Order'...")
        # Pass driver explicitly if needed by the function signature
        click_see_service_order(wait)

        print(f"  Step 3: Navigating to Quotation Tab...")
        # Pass driver explicitly if needed by the function signature
        navigate_to_quotation_tab(driver, wait)

        print(f"  Step 4: Extracting Quotation Details...")
        # quotation_data = extract_quotation_details(driver, wait, service_order)
        # quotations_details.append(quotation_data)  # Append details

        print(f"  Step 5: Navigating to Documents Tab...")
        # Pass driver explicitly if needed by the function signature
        navigate_to_documents_tab(wait)

        print(f"  Step 6: Extracting Document Links...")
        all_docs = extract_document_links(driver, wait)

        print(f"  Step 7: Filtering Report Documents...")
        filtered_list = filter_report_documents(all_docs)

        print(f"  Step 8: Copying {len(filtered_list)} documents to Datalake...")
        copied_files_uris = copy_reso_documents_to_datalake(
            filtered_list, service_order
        )
        processed_docs_info.append(
            {  # Store info about copied files per SO
                "service_order": service_order,
                "filtered_count": len(filtered_list),
                "copied_uris": copied_files_uris,
            }
        )

        print(f"  Step 9: Closing Service Order View...")
        # Pass driver explicitly if needed by the function signature
        close_service_order_view(wait)

        print(
            f"--- Finished processing service order {i+1} of {total_orders}: {service_order} ---"
        )

    except Exception as e:
        print(
            f"!!! ERROR processing service order {service_order} (item {i+1}): {e} !!!"
        )
        print("Attempting to close view and continue...")
        # Attempt to close the view to reset for the next iteration
        try:
            close_service_order_view(wait)
        except Exception as e_close:
            print(f"  Could not close view after error: {e_close}")
            # Consider more drastic recovery like refreshing the page or restarting driver if errors persist
        # Continue to the next service order in the loop
        continue

Found 813 service orders to process.

--- Processing service order 1 of 813: 14108793 ---
  Step 1: Searching for SO...
Locating the service order input field (codigoBuscar)...
Entering service order: 14108793
!!! ERROR processing service order 14108793 (item 1): Message: element not interactable
  (Session info: chrome=135.0.7049.42)
Stacktrace:
	GetHandleVerifier [0x00007FF701121F55+78133]
	GetHandleVerifier [0x00007FF701121FB0+78224]
	(No symbol) [0x00007FF700EE8FEC]
	(No symbol) [0x00007FF700F345F5]
	(No symbol) [0x00007FF700F6715A]
	(No symbol) [0x00007FF700F31A16]
	(No symbol) [0x00007FF700F67370]
	(No symbol) [0x00007FF700F8F21F]
	(No symbol) [0x00007FF700F66F33]
	(No symbol) [0x00007FF700F30358]
	(No symbol) [0x00007FF700F310C3]
	GetHandleVerifier [0x00007FF7013EBA8D+3001453]
	GetHandleVerifier [0x00007FF7013E5E72+2977874]
	GetHandleVerifier [0x00007FF70140497D+3103581]
	GetHandleVerifier [0x00007FF70113C7EA+186826]
	GetHandleVerifier [0x00007FF7011443FF+218591]
	GetHandleVerif

In [ ]:
service_orders = pl.read_parquet(
    r"C:\Users\vales\DataspellProjects\workspace\component_status.parquet"
)["service_order"].to_list()